In [ ]:
from pynq import Overlay
from pynq.lib import AxiGPIO
import time
import numpy as np
%matplotlib inline
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# 載入 Overlay
overlay = Overlay('rgb2yuv_sc.bit')

In [ ]:
# 取得 AXI GPIO
gpio_r   = AxiGPIO(overlay.ip_dict['axi_gpio_R']).channel1
gpio_g   = AxiGPIO(overlay.ip_dict['axi_gpio_G']).channel1
gpio_b   = AxiGPIO(overlay.ip_dict['axi_gpio_B']).channel1
gpio_y   = AxiGPIO(overlay.ip_dict['axi_gpio_Y']).channel1
gpio_u   = AxiGPIO(overlay.ip_dict['axi_gpio_U']).channel1
gpio_v   = AxiGPIO(overlay.ip_dict['axi_gpio_V']).channel1
gpio_rst = AxiGPIO(overlay.ip_dict['axi_gpio_rst']).channel1

mask = 0xffffffff

In [ ]:
# 讀入圖片
img = Image.open("mountain256.bmp").resize((256, 256)).convert("RGB")
r, g, b = img.split()
r = np.array(r).flatten()
g = np.array(g).flatten()
b = np.array(b).flatten()
plt.imshow(img, cmap='gray')
plt.title("RGB")
plt.show()

In [ ]:
# 建立輸出 buffer
sc_y = np.zeros(256*256)
sc_u = np.zeros(256*256)
sc_v = np.zeros(256*256)

In [ ]:
# Reset IP
gpio_rst.write(0, mask)
time.sleep(0.0001)
gpio_rst.write(1, mask)

In [ ]:
sc_start = time.time()
for i in range(256*256):
    gpio_r.write(int(r[i]), mask) # R
    gpio_g.write(int(g[i]), mask) # G
    gpio_b.write(int(b[i]), mask) # B
    #time.sleep(0.0001)
    sc_y[i] = gpio_y.read() # Y
    sc_u[i] = gpio_u.read() # U
    sc_v[i] = gpio_v.read() # V
sc_end = time.time()

In [ ]:
# reshape
sc_y_reshape = sc_y.reshape(256, 256)
sc_u_reshape = sc_u.reshape(256, 256)
sc_v_reshape = sc_v.reshape(256, 256)

In [ ]:
print("Systemc ip spen %f sec" %(sc_end-sc_start)) #顯示圖片
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.imshow(sc_y_reshape, cmap='gray')
plt.title("SystemC HLS Y")

plt.subplot(1, 3, 2)
plt.imshow(sc_u_reshape, cmap='gray')
plt.title("SystemC HLS U")

plt.subplot(1, 3, 3)
plt.imshow(sc_v_reshape, cmap='gray')
plt.title("SystemC HLS V")
plt.show()